<H1>Applied data science capston assignment 3 West Coast Vacation</H1>

This is to find the best city for business start

In [10]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [25]:

address_sf = 'San Francisco, CA, US'

geolocator_sf = Nominatim(user_agent="sf_explorer")
location_sf = geolocator_sf.geocode(address_sf)
latitude_sf = location_sf.latitude
longitude_sf = location_sf.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude_sf, longitude_sf))

The geograpical coordinate of San Francisco are 37.7792808, -122.4192363.


In [26]:
address_la = 'Los Angeles, CA'

geolocator_la = Nominatim(user_agent="la_explorer")
location_la = geolocator_la.geocode(address_la)
latitude_la = location_la.latitude
longitude_la = location_la.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude_la, longitude_la))

The geograpical coordinate of Los Angeles are 34.0536834, -118.2427669.


In [27]:
address_sd = 'San Diego, CA'

geolocator_sd = Nominatim(user_agent="sd_explorer")
location_sd = geolocator_sd.geocode(address_sd)
latitude_sd = location_sd.latitude
longitude_sd = location_sd.longitude
print('The geograpical coordinate of San Diego are {}, {}.'.format(latitude_sd, longitude_sd))

The geograpical coordinate of San Diego are 32.7174209, -117.1627714.


<h1>Explore the neighbors</h1>

In [12]:
from bs4 import BeautifulSoup
import requests

In [28]:

source_sf = requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_San_Francisco').text
source_la = requests.get('https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_of_Los_Angeles').text
source_sd = requests.get('https://en.wikipedia.org/wiki/List_of_communities_and_neighborhoods_of_San_Diego').text

In [29]:
soup_sf = BeautifulSoup(source_sf, 'lxml')




In [30]:
sf_hood_list = soup_sf.find_all('h2')

In [31]:

tag_list =[]
for tag in sf_hood_list:
    tag_list.append(tag.text)


In [32]:
tag_list = tag_list[1:-4]

In [33]:
sf_list = []
for item in tag_list:
    sf_list.append(item[0:-6])

In [34]:
sf_coordinates = pd.DataFrame(sf_list, columns =['Neighbourhood'])

sf_coordinates['Latitude']=0
sf_coordinates['Longitude']=0



In [35]:

for item in sf_coordinates['Neighbourhood']:
    address = '{}, San Francisco, CA, US'.format(item)
    geolocator_sf = Nominatim(user_agent="sf_explorer")
    location_sf = geolocator_sf.geocode(address)
    if location_sf is not None:
        sf_coordinates['Latitude'].loc[sf_coordinates['Neighbourhood']==item] = location_sf.latitude
        sf_coordinates['Longitude'].loc[sf_coordinates['Neighbourhood']==item] = location_sf.longitude


In [36]:
sf_coordinates.head()

,Neighbourhood,Latitude,Longitude
0,Alamo Square,37.776357,-122.434694
1,Anza Vista,37.780836,-122.443149
2,Ashbury Heights,0.000000,0.000000
3,Balboa Park,37.724949,-122.444805
4,Balboa Terrace,0.000000,0.000000


drops the areas not in san_francisco

In [37]:

sf_coordinates = sf_coordinates[sf_coordinates['Latitude']!=0].iloc[:-1, :]
sf_coordinates

,Neighbourhood,Latitude,Longitude
0,Alamo Square,37.776357,-122.434694
1,Anza Vista,37.780836,-122.443149
3,Balboa Park,37.724949,-122.444805
5,Bayview,37.728889,-122.392500
6,Belden Place,37.791744,-122.403886
7,Bernal Heights,37.741001,-122.414214
8,Buena Vista,37.806532,-122.420648
10,Castro,37.760856,-122.434957
13,China Basin,37.776330,-122.391839
14,Chinatown,37.794301,-122.406376


 we got it! Let's move on to Los Angeles

In [38]:
soup_la = BeautifulSoup(source_la, 'lxml')
la_hood_list = soup_la.find_all('li',attrs= {'class':None, 'id': None})
tag_list = []
for hood in la_hood_list:
    tag_list.append(hood.text)
tag_list = tag_list[:-40]
clean_la_list = []
for tag in tag_list:
    clean_la_list.append(tag.split('[')[0])
    

Now that we have a list of LA neighbourhoods we can put them in a dataframe and then find their coordinates like we did with San Francisco

In [ ]:
la_coordinates = pd.DataFrame(clean_la_list, columns = ['Neighbourhood'])

In [ ]:
la_coordinates['Latitude']=0
la_coordinates['Longitude']=0
for item in la_coordinates['Neighbourhood']:
    address = '{}, Los Angeles, CA, US'.format(item)
    geolocator_la = Nominatim(user_agent="la_explorer")
    location_la = geolocator_la.geocode(address)
    if location_la is not None:
        la_coordinates['Latitude'].loc[la_coordinates['Neighbourhood']==item] = location_la.latitude
        la_coordinates['Longitude'].loc[la_coordinates['Neighbourhood']==item] = location_la.longitude
        
la_coordinates = la_coordinates[la_coordinates['Latitude']!=0]
la_coordinates

Getting location data for San Diego

In [ ]:
soup_sd = BeautifulSoup(source_sd, 'lxml')
sd_hood_list = soup_sd.find_all('li', attrs = {'class':None, 'id':None})
tag_list = []
for hood in sd_hood_list:
    tag_list.append(hood.text)

In [ ]:
cleaner_list = tag_list[-139:-22]

In [ ]:
sd_list = []
for entry in cleaner_list:
    if '(' in entry:
        sd_list.append(entry.split('(')[0])
    elif ')' in entry:
        sd_list.append(entry.split(')')[0])
    else:
        sd_list.append(entry)

We can now create the dataframe and get coordinates

In [ ]:
sd_coordinates = pd.DataFrame(sd_list, columns = ['Neighbourhood'])

In [ ]:
d_coordinates['Latitude']=0
sd_coordinates['Longitude']=0

In [ ]:
for item in sd_coordinates['Neighbourhood']:
    address = '{}, San Diego, CA, US'.format(item)
    geolocator_sd = Nominatim(user_agent="sd_explorer")
    location_sd = geolocator_sd.geocode(address)
    if location_sd is not None:
        sd_coordinates['Latitude'].loc[sd_coordinates['Neighbourhood']==item] = location_sd.latitude
        sd_coordinates['Longitude'].loc[sd_coordinates['Neighbourhood']==item] = location_sd.longitude

In [ ]:
sd_coordinates = sd_coordinates[sd_coordinates['Latitude']!=0]
sd_coordinates

Getting the Venues

In [ ]:
sf_coordinates['City']='San Francisco'
la_coordinates['City']='Los Angeles'
sd_coordinates['City']='San Diego'

In [ ]:
all_cities = pd.concat([sf_coordinates, la_coordinates, sd_coordinates])

In [ ]:
all_cities.reset_index(drop=True)

In [ ]:
CLIENT_ID = 'U2NNANEUQUQ2NFUN2N25XHS1A4GLC0J15D2LIISWSIYLEKF0' # your Foursquare ID
CLIENT_SECRET = 'GY5UVEHHCPD5DUXCTHI0MXWVDO5ND4OWAO3SMMMWYX2NZNVT' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
west_coast_venues = getNearbyVenues(names=all_cities['Neighbourhood'],
                                   latitudes=all_cities['Latitude'],
                                   longitudes=all_cities['Longitude']
                                  )

In [ ]:
print(west_coast_venues.shape)
west_coast_venues.head()

In [ ]:
print('There are {} uniques categories.'.format(len(west_coast_venues['Venue Category'].unique())))

In [ ]:
west_coast_venues_cities = west_coast_venues.merge(all_cities, how='left', left_on = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude'], right_on=['Neighbourhood', 'Latitude', 'Longitude'])

In [ ]:

west_coast_venues_cities= west_coast_venues_cities.drop(['Neighbourhood', 'Latitude', 'Longitude'], axis=1)

In [ ]:
west_coast_venues_cities.shape

In [ ]:
west_coast_venues_cities.head()